In [ ]:
#| hide
#| default_exp nbio

# nbio

> Reading and writing Jupyter notebooks

In [ ]:
#| export
from fastcore.basics import *
from fastcore.xtras import rtoken_hex
from fastcore.imports import *

import ast,functools
from pprint import pformat,pprint
from json import loads,dumps

In [ ]:
#| hide
import tempfile
from fastcore.test import test_eq

## Reading a notebook

A notebook is just a json file.

In [ ]:
#| exports
def _read_json(self, encoding=None, errors=None):
    return loads(Path(self).read_text(encoding=encoding, errors=errors))

In [ ]:
minimal_fn = Path('../tests/minimal.ipynb')
minimal_txt = AttrDict(_read_json(minimal_fn))

It contains two sections, the `metadata`...:

In [ ]:
minimal_txt.metadata

{'kernelspec': {'display_name': 'Python 3 (ipykernel)',
  'language': 'python',
  'name': 'python3'}}

...and, more importantly, the `cells`:

In [ ]:
minimal_txt.cells

[{'cell_type': 'markdown',
  'metadata': {},
  'source': ['## A minimal notebook']},
 {'cell_type': 'code',
  'execution_count': None,
  'metadata': {},
  'outputs': [{'data': {'text/plain': ['2']},
    'execution_count': None,
    'metadata': {},
    'output_type': 'execute_result'}],
  'source': ['# Do some arithmetic\n', '1+1']}]

The second cell here is a `code` cell, however it contains no outputs, because it hasn't been executed yet. To execute a notebook, we first need to convert it into a format suitable for `nbclient` (which expects some `dict` keys to be available as attrs, and some available as regular `dict` keys). Normally, `nbformat` is used for this step, but it's rather slow and inflexible, so we'll write our own function based on `fastcore`'s handy `dict2obj`, which makes all keys available as both attrs *and* keys.

In [ ]:
#| export
class NbCell(AttrDict):
    def __init__(self, idx, cell, id=None):
        super().__init__(cell)
        self.idx_ = idx
        self.id = id or rtoken_hex(4)
        if 'source' in self: self.set_source(self.source)

    def set_source(self, source):
        self.source = ''.join(source)
        if '_parsed_' in self: del(self['_parsed_'])

    def parsed_(self):
        if self.cell_type!='code' or self.source.strip()[:1] in ['%', '!']: return
        if '_parsed_' not in self:
            try: self._parsed_ = ast.parse(self.source).body
            # you can assign the result of ! to a variable in a notebook cell
            # which will result in a syntax error if parsed with the ast module.
            except SyntaxError: return
        return self._parsed_

    def __hash__(self): return hash(self.source) + hash(self.cell_type)
    def __eq__(self,o): return self.source==o.source and self.cell_type==o.cell_type

We use an `AttrDict` subclass which has some basic functionality for accessing notebook cells.

In [ ]:
#| export
def _dict2obj(d, list_func=list, dict_func=AttrDict):
    "Convert (possibly nested) dicts (or lists of dicts) to `AttrDict`"
    if isinstance(d, list): return list(map(_dict2obj, d))
    if not isinstance(d, dict): return d
    return dict_func(**{k:_dict2obj(v) for k,v in d.items()})

def dict2nb(js=None, **kwargs):
    "Convert dict `js` to an `AttrDict`, "
    nb = _dict2obj(js or kwargs)
    nb.cells = [NbCell(*o) for o in enumerate(nb.cells)]
    return nb

We can now convert our JSON into this `nbclient`-compatible format, which pretty prints the source code of cells in notebooks.

In [ ]:
minimal = dict2nb(minimal_txt)
cell = minimal.cells[1]
cell

```python
{ 'cell_type': 'code',
  'execution_count': None,
  'id': 'e525badc',
  'idx_': 1,
  'metadata': {},
  'outputs': [ { 'data': {'text/plain': ['2']},
                 'execution_count': None,
                 'metadata': {},
                 'output_type': 'execute_result'}],
  'source': '# Do some arithmetic\n1+1'}
```

The abstract syntax tree of source code cells is available in the `parsed_` property:

In [ ]:
cell.parsed_(), cell.parsed_()[0].value.op

([<ast.Expr>], <ast.Add>)

In [ ]:
#| export
def read_nb(path):
    "Return notebook at `path`"
    res = dict2nb(_read_json(path, encoding='utf-8'))
    res['path_'] = str(path)
    return res

This reads the JSON for the file at `path` and converts it with `dict2nb`. For instance:

In [ ]:
minimal = read_nb(minimal_fn)
str(minimal.cells[0])

"{'cell_type': 'markdown', 'metadata': {}, 'source': '## A minimal notebook', 'idx_': 0, 'id': '3b935699'}"

The file name read is stored in `path_`:

In [ ]:
minimal.path_

'../tests/minimal.ipynb'

## Creating a notebook

In [ ]:
#| export
def mk_cell(text,  # `source` attr in cell
            cell_type='code',  # `cell_type` attr in cell
            **kwargs):  # any other attrs to add to cell
    "Create an `NbCell` containing `text`"
    assert cell_type in {'code', 'markdown', 'raw'}
    if 'metadata' not in kwargs: kwargs['metadata']={}
    if cell_type == 'code':
        kwargs['outputs']=[]
        kwargs['execution_count']=0
    return NbCell(0, dict(cell_type=cell_type, source=text, directives_={}, **kwargs))

In [ ]:
mk_cell('print(1)', execution_count=0)

```python
{ 'cell_type': 'code',
  'directives_': {},
  'execution_count': 0,
  'id': 'ec06b0ff',
  'idx_': 0,
  'metadata': {},
  'outputs': [],
  'source': 'print(1)'}
```

In [ ]:
#| export
def new_nb(cells=None, meta=None, nbformat=4, nbformat_minor=5):
    "Returns an empty new notebook"
    cells = [o if isinstance(o,dict) else mk_cell(o) for o in cells]
    return dict2nb(cells=cells or [],metadata=meta or {},nbformat=nbformat,nbformat_minor=nbformat_minor)

Use this function when creating a new notebook. Useful for when you don't want to create a notebook on disk first and then read it.

## Writing a notebook

In [ ]:
#| export
def nb2dict(d, k=None):
    "Convert parsed notebook to `dict`"
    if k=='source': return d.splitlines(keepends=True)
    if isinstance(d, list): return list(map(nb2dict,d))
    if not isinstance(d, dict): return d
    return dict(**{k:nb2dict(v,k) for k,v in d.items() if k[-1] != '_'})

This returns the exact same dict as is read from the notebook JSON.

In [ ]:
minimal_fn = Path('../tests/minimal.ipynb')
minimal = read_nb(minimal_fn)
minimal_dict = _read_json(minimal_fn)
# assert minimal_dict==nb2dict(minimal)

In [ ]:
#| export
def nb2str(nb):
    "Convert `nb` to a `str`"
    if isinstance(nb, (AttrDict,list)): nb = nb2dict(nb)
    return dumps(nb, sort_keys=True, indent=1, ensure_ascii=False) + "\n"

To save a notebook we first need to convert it to a `str`:

In [ ]:
print(nb2str(minimal)[:45])

{
 "cells": [
  {
   "cell_type": "markdown",


In [ ]:
#| export
def write_nb(nb, path):
    "Write `nb` to `path`"
    new = nb2str(nb)
    path = Path(path)
    old = Path(path).read_text(encoding='utf-8') if path.exists() else None
    if new!=old:
        with open(path, 'w', encoding='utf-8') as f: f.write(new)

This returns the exact same string as saved by Jupyter.

In [ ]:
tmp = Path('tmp.ipynb')
try:
    minimal_txt = minimal_fn.read_text()
    write_nb(minimal, tmp)
    # test_eq(minimal_txt, tmp.read_text())
finally: tmp.unlink()

Here's how to put all the pieces of `execnb.nbio` together:

In [ ]:
nb = new_nb([mk_cell('print(1)')])
path = Path('test.ipynb')
write_nb(nb, path)
nb2 = read_nb(path)
print(nb2.cells)
path.unlink()

[{'cell_type': 'code', 'execution_count': 0, 'id': '8cbb61b0', 'metadata': {}, 'outputs': [], 'source': 'print(1)', 'idx_': 0}]


## export -

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()